In [2]:
# Understand LSTM gates
#1 the forget gate calculation
import numpy as np 
np.random.seed(0) # fix random output

def sigmoid(x):
    return 1/(1+ np.exp(-x))

# Simulating dummy values for previous state and current input
h_prev = np.random.randn(3,1)
x = np.random.randn(5,1)

# initialize Wf, Uf
Wf = np.random.randn(3,5) # n_h = 3, n_x = 5; 
Uf = np.random.randn(3,3)

# forget gate
f = sigmoid(np.matmul(Wf,x) + np.matmul(Uf, h_prev))
f  # size is [n_h,1]

array([[0.45930054],
       [0.97661676],
       [0.99403442]])

In [3]:
#2. the input gate (i_t)
Wi = np.random.randn(3,5) # n_h = 3, n_x = 5
Ui = np.random.randn(3,3) # n_h = 3

i = sigmoid( np.matmul(Wi,x) + np.matmul(Ui, h_prev))
i

array([[0.00762368],
       [0.39184172],
       [0.17027909]])

In [4]:
#3. The candidate cell state
Wc = np.random.randn(3,5)
Uc = np.random.randn(3,3)

C_candidate = np.tanh(np.matmul(Wc,x)+np.matmul(Uc,h_prev))
C_candidate

array([[ 0.51233992],
       [-0.67747899],
       [-0.99555958]])

In [6]:
#4. Cell state Update 
c_prev = np.random.randn(3,1)
c_new = np.multiply(f, c_prev) + np.multiply(i,C_candidate)
c_new

array([[-0.70169163],
       [ 1.1879875 ],
       [ 1.71505613]])

In [11]:
#5. Ouput gate  and Current Activation 
# output gate o
Wo = np.random.randn(3,5)
Uo = np.random.randn(3,3)
o = np.tanh(np.matmul(Wo, x) + np.matmul(Uo, h_prev))
o

h_new = np.multiply(o, np.tanh(c_new))
h_new

array([[ 0.58782959],
       [-0.82993761],
       [ 0.74022281]])

In [60]:
#### Exercise 27. LSTM model to classify Email as Spam or not Spam #####
#1. 
import pandas as pd
import numpy as np
from keras.models import Model, Sequential
from keras.layers import GRU, LSTM, Dense, Embedding

from keras. preprocessing.text import Tokenizer

#Read file 
df = pd.read_csv("spam.csv",encoding="latin")
# df.head()
#2 Only need Text and Label
df = df[["v1","v2"]]
# df.head()
#3. Chec label distribution 
df["v1"].value_counts()

#4. Map label distriution to 0/1 so that it can be fed to classifier 
label_map = {"ham":0, "spam":1}
X = df["v2"].values
y = df["v1"].map(label_map).values # map() => t create label, values() to return the values as nympy.ndarray without getting axes labels

len(y)

5572

In [108]:
#5. restrict the maximum number of tokens to be generated the 100 most frequent words 
# intialize a Tokenizer to assign an interger value to each word being used in the corpus
Max_words = 100 # Max >=6000
MyToken = Tokenizer(nb_words = Max_words, lower = True, split=" ")
MyToken.fit_on_texts(X)
text_tokenized = MyToken.texts_to_sequences(X) # Only the words in text belongs to top 100 most frequent words get to be assigned an integer. Ex: 1 sequences X[0] has 20 words, only 6 indices in tokenized representation.
# # max_len = max([len(i) for i in text_tokenized])
# print(max_len)
print(max(text_tokenized, key=len))
print(len(max(text_tokenized, key=len)))
max_pos = text_tokenized.index(max(text_tokenized, key=len))
print(max_pos)
print(text_tokenized[1084])

[12, 10, 5, 66, 36, 1, 20, 1, 79, 63, 10, 95, 32, 5, 8, 11, 1, 5, 65, 7, 14, 36, 95, 32, 67, 63, 1, 66, 33, 32, 57, 45, 11, 11, 11, 33, 32, 2, 1, 12, 33, 51, 11, 7, 97, 51, 1, 33, 32, 2, 36, 12, 1, 33, 32, 8, 66, 45, 1, 33, 32, 5, 12, 66, 33, 32, 45, 1, 98, 17, 2, 20, 11, 9, 5, 18, 5, 1, 33, 32, 12, 66, 33, 32, 45, 1, 44, 7, 44, 33, 32, 5, 57, 33, 31, 7, 12, 5, 65, 95, 9, 36, 10, 1, 55, 2, 4, 33, 99]
109
1084
[12, 10, 5, 66, 36, 1, 20, 1, 79, 63, 10, 95, 32, 5, 8, 11, 1, 5, 65, 7, 14, 36, 95, 32, 67, 63, 1, 66, 33, 32, 57, 45, 11, 11, 11, 33, 32, 2, 1, 12, 33, 51, 11, 7, 97, 51, 1, 33, 32, 2, 36, 12, 1, 33, 32, 8, 66, 45, 1, 33, 32, 5, 12, 66, 33, 32, 45, 1, 98, 17, 2, 20, 11, 9, 5, 18, 5, 1, 33, 32, 12, 66, 33, 32, 45, 1, 44, 7, 44, 33, 32, 5, 57, 33, 31, 7, 12, 5, 65, 95, 9, 36, 10, 1, 55, 2, 4, 33, 99]


In [98]:
# Max Sequence Length of 50 words per/sequence, and pad those sequences shorter than this lengh.  
Max_len = 50
from keras.preprocessing import sequence
sequences = sequence.pad_sequences(text_tokenized, maxlen=Max_len)
sequences

array([[ 0,  0,  0, ..., 89, 67, 58],
       [ 0,  0,  0, ...,  0, 46,  6],
       [ 0,  0,  0, ...,  2,  2, 73],
       ...,
       [ 0,  0,  0, ..., 12, 20, 23],
       [ 0,  0,  0, ...,  2, 12, 47],
       [ 0,  0,  0, ..., 61,  2, 61]])

In [111]:
sequences[1084] #  this one got truncated

array([33, 32,  5, 12, 66, 33, 32, 45,  1, 98, 17,  2, 20, 11,  9,  5, 18,
        5,  1, 33, 32, 12, 66, 33, 32, 45,  1, 44,  7, 44, 33, 32,  5, 57,
       33, 31,  7, 12,  5, 65, 95,  9, 36, 10,  1, 55,  2,  4, 33, 99])

In [113]:
model = Sequential()
model.add(Embedding(Max_words, 20, input_length=Max_len))
model.add(LSTM(64))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 20)            2000      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                21760     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 23,825
Trainable params: 23,825
Non-trainable params: 0
_________________________________________________________________


In [115]:
model.fit(sequences,y, batch_size= 128, epochs= 15, validation_split=0.2)

Train on 4457 samples, validate on 1115 samples
Epoch 1/15
4457/4457 [==============================] - 1s 323us/step - loss: 0.0887 - accuracy: 0.9704 - val_loss: 0.1009 - val_accuracy: 0.9641
Epoch 2/15
4457/4457 [==============================] - 1s 313us/step - loss: 0.0870 - accuracy: 0.9726 - val_loss: 0.1061 - val_accuracy: 0.9677
Epoch 3/15
4457/4457 [==============================] - 1s 311us/step - loss: 0.0872 - accuracy: 0.9715 - val_loss: 0.1014 - val_accuracy: 0.9650
Epoch 4/15
4457/4457 [==============================] - 1s 317us/step - loss: 0.0851 - accuracy: 0.9731 - val_loss: 0.1007 - val_accuracy: 0.9650
Epoch 5/15
4457/4457 [==============================] - 1s 302us/step - loss: 0.0832 - accuracy: 0.9735 - val_loss: 0.1010 - val_accuracy: 0.9650
Epoch 6/15
4457/4457 [==============================] - 1s 313us/step - loss: 0.0827 - accuracy: 0.9742 - val_loss: 0.1027 - val_accuracy: 0.9695
Epoch 7/15
4457/4457 [==============================] - 1s 307us/step - loss

In [132]:
inp_test_seq = "WINNER! U win a 500 prize reward & free entry to FA cup final tickets! Text FA to 34212 to receive award"
inp_test_seq2 = "asdfadsfasdf Free entry in 2 a wkly asdfadf comp to win FA asdfadf Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's asdfadsf"

inp_test_seq3 = "Thanks for your subscription to Ringtone UK your mobile will be charged �5/month Please confirm by replying YES or NO. If you reply NO you will not be charged. Thank you for reading message. Call me back immediately"
test_sequences = MyToken.texts_to_sequences(np.array([inp_test_seq2]))
test_sequences_vec = sequence.pad_sequences(test_sequences, maxlen= Max_len)

model.predict(test_sequences_vec)

array([[0.9985812]], dtype=float32)

In [122]:
test_sequences = MyToken.texts_to_sequences(np.array([inp_test_seq2]))
test_sequences_vec = sequence.pad_sequences(test_sequences, maxlen= Max_len)
model.predict(test_sequences_vec)

array([[0.9985812]], dtype=float32)

In [134]:
# Activity 9 Extra :compare with GRU
from keras.layers import GRU
GRUmodel = Sequential()
GRUmodel.add(Embedding(Max_words, 20, input_length=Max_len))
GRUmodel.add(GRU(64))
GRUmodel.add(Dense(1, activation="sigmoid"))
GRUmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
GRUmodel.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 20)            2000      
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                16320     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 18,385
Trainable params: 18,385
Non-trainable params: 0
_________________________________________________________________


In [136]:
GRUmodel.fit(sequences,y, batch_size= 128, epochs= 15, validation_split=0.2)

Train on 4457 samples, validate on 1115 samples
Epoch 1/15
4457/4457 [==============================] - 2s 363us/step - loss: 0.5396 - accuracy: 0.8438 - val_loss: 0.3390 - val_accuracy: 0.8700
Epoch 2/15
4457/4457 [==============================] - 1s 278us/step - loss: 0.2841 - accuracy: 0.8746 - val_loss: 0.1869 - val_accuracy: 0.9247
Epoch 3/15
4457/4457 [==============================] - 1s 278us/step - loss: 0.1460 - accuracy: 0.9518 - val_loss: 0.1237 - val_accuracy: 0.9596
Epoch 4/15
4457/4457 [==============================] - 1s 274us/step - loss: 0.1125 - accuracy: 0.9639 - val_loss: 0.1111 - val_accuracy: 0.9641
Epoch 5/15
4457/4457 [==============================] - 1s 275us/step - loss: 0.1018 - accuracy: 0.9654 - val_loss: 0.1032 - val_accuracy: 0.9686
Epoch 6/15
4457/4457 [==============================] - 1s 277us/step - loss: 0.1007 - accuracy: 0.9681 - val_loss: 0.1247 - val_accuracy: 0.9578
Epoch 7/15
4457/4457 [==============================] - 1s 284us/step - loss

In [139]:
inp_test_seq = "WINNER! U win a 500 prize reward & free entry to FA cup final tickets! Text FA to 34212 to receive award"
inp_test_seq2 = "asdfadsfasdf Free entry in 2 a wkly asdfadf comp to win FA asdfadf Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's asdfadsf"

inp_test_seq3 = "Thanks for your subscription to Ringtone UK your mobile will be charged �5/month Please confirm by replying YES or NO. If you reply NO you will not be charged. Thank you for reading message. Call me back immediately"
test_sequences = MyToken.texts_to_sequences(np.array([inp_test_seq3]))
test_sequences_vec = sequence.pad_sequences(test_sequences, maxlen= Max_len)

GRUmodel.predict(test_sequences_vec)

array([[0.98544204]], dtype=float32)

In [140]:
# Activity 9 compare with SimpleRNN
from keras.layers import SimpleRNN
RNNmodel = Sequential()
RNNmodel.add(Embedding(Max_words, 20, input_length=Max_len))
RNNmodel.add(SimpleRNN(64))
RNNmodel.add(Dense(1, activation="sigmoid"))
RNNmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
RNNmodel.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 20)            2000      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                5440      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 7,505
Trainable params: 7,505
Non-trainable params: 0
_________________________________________________________________


In [141]:
RNNmodel.fit(sequences,y, batch_size= 128, epochs= 15, validation_split=0.2)

Train on 4457 samples, validate on 1115 samples
Epoch 1/15
4457/4457 [==============================] - 1s 139us/step - loss: 0.4314 - accuracy: 0.8427 - val_loss: 0.3763 - val_accuracy: 0.8700
Epoch 2/15
4457/4457 [==============================] - 0s 83us/step - loss: 0.3738 - accuracy: 0.8649 - val_loss: 0.3096 - val_accuracy: 0.8709
Epoch 3/15
4457/4457 [==============================] - 0s 80us/step - loss: 0.2587 - accuracy: 0.9006 - val_loss: 0.1876 - val_accuracy: 0.9300
Epoch 4/15
4457/4457 [==============================] - 0s 78us/step - loss: 0.1600 - accuracy: 0.9455 - val_loss: 0.1311 - val_accuracy: 0.9561
Epoch 5/15
4457/4457 [==============================] - 0s 95us/step - loss: 0.1231 - accuracy: 0.9567 - val_loss: 0.1120 - val_accuracy: 0.9632
Epoch 6/15
4457/4457 [==============================] - 0s 90us/step - loss: 0.1113 - accuracy: 0.9619 - val_loss: 0.1101 - val_accuracy: 0.9632
Epoch 7/15
4457/4457 [==============================] - 0s 97us/step - loss: 0.12

In [145]:
inp_test_seq = "WINNER! U win a 500 prize reward & free entry to FA cup final tickets! Text FA to 34212 to receive award"
inp_test_seq2 = "asdfadsfasdf Free entry in 2 a wkly asdfadf comp to win FA asdfadf Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's asdfadsf"

inp_test_seq3 = "Thanks for your subscription to Ringtone UK your mobile will be charged �5/month Please confirm by replying YES or NO. If you reply NO you will not be charged. Thank you for reading message. Call me back immediately"
test_sequences = MyToken.texts_to_sequences(np.array([inp_test_seq3]))
test_sequences_vec = sequence.pad_sequences(test_sequences, maxlen= Max_len)

RNNmodel.predict(test_sequences_vec)

array([[0.92302144]], dtype=float32)